In [2]:
from tkinter import *
from tkinter import ttk
import tkinter.font as tkFont
from tkinter import font

In [3]:
from movie import Movie

In [4]:
from parse import parse_imdb_data

In [5]:
import os
import pickle

In [6]:
if os.path.isfile("movies.p"):
    movies = pickle.load(open("movies.p", "rb"))
else:
    movies = parse_imdb_data()
    pickle.dump(movies, open( "movies.p", "wb" ))

In [19]:
m_ = movies
#m_=list(filter(lambda m: m.year>2000, m_))
m_=list(filter(lambda m: m.num_ratings>10000, m_))
#m_=list(filter(lambda m: m.mean_rating>8.5, m_))
len(m_)

8000

In [20]:
def sortby(tree, col, descending):
    """sort tree contents when a column header is clicked on"""
    # grab values to sort
    data = [(tree.set(child, col), child) \
        for child in tree.get_children('')]
    # if the data to be sorted is numeric change to float
    #data =  change_numeric(data)
    # now sort the data in place
    if(data[0][0].isnumeric()): # FIX: doesn't work for float
        data = [(float(d[0]), d[1]) for d in data]
    data.sort(reverse=descending)
    for ix, item in enumerate(data):
        tree.move(item[1], '', ix)
    # switch the heading so it will sort in the opposite direction
    tree.heading(col, command=lambda col=col: sortby(tree, col, \
        int(not descending)))

In [21]:
min_year = min_rating_count = min_mean_rating = float('inf')
max_year = max_rating_count = max_mean_rating = float('-inf')
for m in m_:
    min_year = min((min_year, m.year))
    max_year = max((max_year, m.year))
    min_rating_count = min((min_rating_count, m.num_ratings))
    max_rating_count = max((max_rating_count, m.num_ratings))
    min_mean_rating = min((min_mean_rating, m.mean_rating))
    max_mean_rating = max((max_mean_rating, m.mean_rating))    

In [10]:
if os.path.isfile("ratings.p"):
    ratings = pickle.load(open("ratings.p", "rb"))
else:
    ratings = {}

In [157]:
ratings = {}

In [158]:
root = Tk()
root.title("IMDB MOVIE RECOMMENDER SYSTEM")

mainframe = ttk.Frame(root, padding="3 3 12 12")
mainframe.grid(column=0, row=0, sticky=(N, W, E, S))

filterframe = ttk.Labelframe(mainframe, padding="3 3 12 12", text="Filter")
filterframe.grid(column=0, row=0, sticky=(N, W))
filterframe['borderwidth'] = 2
filterframe['relief'] = 'sunken'


yearfilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="Year")
yearfilterframe.grid(column=0, row=0, sticky=(N, E))
yearfilterframe['borderwidth'] = 2
yearfilterframe['relief'] = 'sunken'
ttk.Label(yearfilterframe, text="MIN").grid(column=0, row=0, sticky="W")
minyear=IntVar()
minyear.set(min_year)
ttk.Entry(yearfilterframe, textvariable=minyear).grid(column=1, row=0, sticky="E")
ttk.Label(yearfilterframe, text="MAX").grid(column=0, row=1, sticky="W")
maxyear=IntVar()
maxyear.set(max_year)
ttk.Entry(yearfilterframe, textvariable=maxyear).grid(column=1, row=1, sticky="E")


numratingfilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="# of Ratings")
numratingfilterframe.grid(column=0, row=1, sticky=(N, E))
numratingfilterframe['borderwidth'] = 2
numratingfilterframe['relief'] = 'sunken'
ttk.Label(numratingfilterframe, text="MIN").grid(column=0, row=0, sticky="E")
minratingcount=IntVar()
minratingcount.set(min_rating_count)
ttk.Entry(numratingfilterframe, textvariable=minratingcount).grid(column=1, row=0, sticky="W")
ttk.Label(numratingfilterframe, text="MAX").grid(column=0, row=1, sticky="E")
maxratingcount=IntVar()
maxratingcount.set(max_rating_count)
ttk.Entry(numratingfilterframe, textvariable=maxratingcount).grid(column=1, row=1, sticky="W")


meanratingfilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="# of Ratings")
meanratingfilterframe.grid(column=0, row=2, sticky=(N, E))
meanratingfilterframe['borderwidth'] = 2
meanratingfilterframe['relief'] = 'sunken'
ttk.Label(meanratingfilterframe, text="MIN").grid(column=0, row=0, sticky="W")
minrating=DoubleVar()
minrating.set(min_mean_rating)
ttk.Entry(meanratingfilterframe, textvariable=minrating).grid(column=1, row=0, sticky="E")
ttk.Label(meanratingfilterframe, text="MAX").grid(column=0, row=1, sticky="W")
maxrating=DoubleVar()
maxrating.set(max_mean_rating)
ttk.Entry(meanratingfilterframe, textvariable=maxrating).grid(column=1, row=1, sticky="E")

phrasefilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="Phrase")
phrasefilterframe.grid(column=0, row=3, sticky="nsew")
phrasefilterframe['borderwidth'] = 2
phrasefilterframe['relief'] = 'sunken'

titlephrase = StringVar()
ttk.Label(phrasefilterframe, text="Title").grid(column=0, row=0, sticky="W")
ttk.Entry(phrasefilterframe, textvariable=titlephrase).grid(column=1, row=0, sticky="nsew")
keywordphrase = StringVar()
ttk.Label(phrasefilterframe, text="Keyword").grid(column=0, row=1, sticky="W")
ttk.Entry(phrasefilterframe, textvariable=keywordphrase).grid(column=1, row=1, sticky="nsew")
genrephrase = StringVar()
ttk.Label(phrasefilterframe, text="Genre").grid(column=0, row=2, sticky="W")
ttk.Entry(phrasefilterframe, textvariable=genrephrase).grid(column=1, row=2, sticky="nsew")

removed_iids = set()

def filter_data(*args):
    remove_iids = set()
    data = data_tree.get_children('')
    #print(data_tree.set(data[0]))    
    for iid in data:
        m = m_[int(iid)]
        # Year
        if m.year < minyear.get():
            remove_iids.add(iid)
        if m.year > maxyear.get():
            remove_iids.add(iid)
        # Rating Count
        if m.num_ratings < minratingcount.get():
            remove_iids.add(iid)
        if m.num_ratings > maxratingcount.get():
            remove_iids.add(iid)
        # MEAN RATING
        if m.mean_rating < minrating.get():
            remove_iids.add(iid)
        if m.mean_rating > maxrating.get():
            remove_iids.add(iid)
        if m.title.find(titlephrase.get()) == -1:
            remove_iids.add(iid)
        
        if genrephrase.get() != "":
            if m.genres == None:
                remove_iids.add(iid)
            else:
                gp = False
                for g in m.genres:
                    if g.find(genrephrase.get()) != -1:
                        gp = True
                        break
                if not gp:
                    remove_iids.add(iid)
        if keywordphrase.get() != "":
            if m.keywords == None:
                remove_iids.add(iid)
            else:
                kp = False
                for k in m.keywords:
                    if k.find(keywordphrase.get()) != -1:
                        kp = True
                        break
                if not kp:
                    remove_iids.add(iid)
        
    #data_tree.delete(list(remove_iids))
    for iid in remove_iids:
        data_tree.delete(iid)
    removed_iids.update(remove_iids)
    #TODO
    # 1. Update the other fields based on the new filtered data
    # 2. There is no way to go back in filtering right now

def reset_filters(*args):
    for iid in removed_iids:
        i = int(iid)
        item = [m_[i].title, m_[i].year, m_[i].num_ratings, m_[i].mean_rating]
        data_tree.insert('', 'end', iid = str(i), values=item)
    sortby(data_tree, 'Title', False)
    removed_iids.clear()
    #TODO Reset the fields to their default values
    
def double_click(e):
    item = data_tree.selection()[0]
    #print("You clicked on", data_tree.set(item))
    #print(m_[int(item)])
    tl = Toplevel(root)
    #ttk.Label(tl, text=data_tree.set(item)).grid(column=0, row=0, sticky="E")
    m = m_[int(item)]
    
    def rate_movie(*args):
        #print(m.title)
        #print(rating.get())
        #ratings[m.title+str(m.year)]=rating.get()
        if rating.get() != "no-opinion":
            ratings[m] = rating.get()
        else:
            ratings.pop(m, None)
        tl.destroy()
    
    ttk.Label(tl, text=m.title + " (" + str(m.year) + ")", font=font.Font(family='Helvetica', size=14, weight='bold')).grid(column=0, row=0, sticky="W")
    if m.genres is not None:
        g = ", ".join(m.genres)
        ttk.Label(tl, text="Genres: " + g).grid(column=0, row=1, sticky="W")      
    if m.keywords is not None:
        #cnames = StringVar(value=countrynames)
        kl= Listbox(tl, listvariable=StringVar(value=m.keywords), height=5)
        kl.grid(column=0, row=2, sticky="nsew")
        vs = ttk.Scrollbar(tl, orient=VERTICAL, command=kl.yview)
        kl.configure(yscrollcommand=vs.set)
        vs.grid(column=1, row=2, sticky="nsew")
        hs = ttk.Scrollbar(tl, orient=HORIZONTAL, command=kl.xview)
        kl.configure(xscrollcommand=hs.set)
        hs.grid(column=0, row=3, sticky="nsew")
    rateframe=ttk.Frame(tl, padding="3 3 12 12")
    rateframe.grid(column=0, row=4, sticky="nsew")        
    rating = StringVar()
    if m in ratings:
        rating.set(ratings[m])
    else:
        rating.set("no-opinion")
    ttk.Radiobutton(rateframe, text='Like', variable=rating, value='like').grid(column=0, row=0, sticky="nsew")
    ttk.Radiobutton(rateframe, text='Dislike', variable=rating, value='dislike').grid(column=1, row=0, sticky="nsew")
    ttk.Radiobutton(rateframe, text='No opinion', variable=rating, value='no-opinion').grid(column=2, row=0, sticky="nsew")
    ttk.Button(rateframe, text="Submit", command=rate_movie).grid(column=1, row=1, sticky = "nsew")
        
    

buttonfilterframe = ttk.Frame(filterframe, padding="3 3 12 12")
buttonfilterframe.grid(column=0, row=4, sticky=(N, E))

ttk.Button(buttonfilterframe, text="Filter", command=filter_data).grid(column=0, row=0, sticky = "W")
ttk.Button(buttonfilterframe, text="Reset", command=reset_filters).grid(column=1, row=0, sticky = "E")

#s = Scale(filterframe, orient=HORIZONTAL, length=200, from_=1900, to=2017 , showvalue=True, label="Minimum Year", command=filter_year)
#s.grid(column=0, row=0, sticky=W)

#s = Scale(filterframe, orient=HORIZONTAL, length=200, from_=0, to=1e3, resolution=1, showvalue=True, label="Minimum # Ratings", command=filter_num_ratings)
#s.grid(column=0, row=1, sticky=W)

#s = Scale(filterframe, orient=HORIZONTAL, length=200, from_=0.0, to=10.0, resolution=0.1, showvalue=True, label="Minimum Mean Rating")
#s.grid(column=0, row=2, sticky=W)




dataframe = ttk.Labelframe(mainframe, padding="3 3 12 12", text="Data")
dataframe.grid(column=1, row=0, sticky="ne")
dataframe['borderwidth'] = 2
dataframe['relief'] = 'sunken'

searchfilterframe = ttk.Frame(dataframe, padding="3 3 12 12")
searchfilterframe['borderwidth'] = 2
searchfilterframe['relief'] = 'sunken'
searchfilterframe.grid(column=0, row=0, sticky="nsew")



# some of the following code is borrowed from https://www.daniweb.com/programming/software-development/threads/350266/creating-table-in-python

data_header = ['Title', 'Year', 'Rating Count', "Mean Rating"]
#data = [[m.title, m.year, m.num_ratings, m.mean_rating] for m in m_]

data_tree = ttk.Treeview(dataframe, columns=data_header, show="headings", selectmode='browse')

vsb = ttk.Scrollbar(dataframe, orient="vertical", command=data_tree.yview)
hsb = ttk.Scrollbar(dataframe, orient="horizontal", command=data_tree.xview)

data_tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)
data_tree.grid(column=0, row=1, sticky='nsew')

data_tree.bind("<Double-1>", double_click)

vsb.grid(column=1, row=1, sticky='ns')
hsb.grid(column=0, row=2, sticky='ew')

dataframe.grid_columnconfigure(0, weight=1)
dataframe.grid_rowconfigure(0, weight=1)


for col in data_header:
    data_tree.heading(col, text=col.title(), command=lambda c=col: sortby(data_tree, c, 0))
    # adjust the column's width to the header string
    data_tree.column(col, width=tkFont.Font().measure(col.title()))
    
for i in range(len(m_)):
    item = [m_[i].title, m_[i].year, m_[i].num_ratings, m_[i].mean_rating]
    data_tree.insert('', 'end', iid = str(i), values=item)
    # adjust column's width if necessary to fit each value
    for ix, val in enumerate(item):
        col_w = tkFont.Font().measure(val)
        if data_tree.column(data_header[ix],width=None)<col_w:
            data_tree.column(data_header[ix], width=col_w)

root.mainloop()

In [379]:
ratings = {}

look_for_genres = set()
look_for_genres.update(['Family'])

look_for_keywords = set()
look_for_keywords.update(['based-on-novel', 'mother-son-relationship'])

for m in m_:
    if m.genres is not None:
        mgs = set()
        mgs.update(m.genres)
        if len(look_for_genres.intersection(mgs)) == len(look_for_genres):
            if m.keywords is not None:
                mks = set()
                mks.update(m.keywords)
                if len(look_for_keywords.intersection(mks)) == len(look_for_keywords):
                    ratings[m] = 'like'

In [380]:
len(ratings)

51

In [381]:
pickle.dump(ratings, open( "ratings.p", "wb" ))

In [382]:
# Vectorize
# #Ratings, z-scored Mean Rating, #Ratings x z-scored meanrating, Genres, keywords
Genres = set()
Keywords = set()
for m in ratings:
    if m.genres is not None:
        for g in m.genres:
            Genres.add(g)
    if m.keywords is not None:
        for k in m.keywords:
            Keywords.add(k)

In [383]:
len(Genres)

18

In [384]:
len(Keywords)

4215

In [385]:
Genres = list(Genres)

In [386]:
Keywords = list(Keywords)

In [387]:
import numpy as np

In [388]:
r = []
for m in movies:
    r.append(m.mean_rating)

In [389]:
r_mean = np.mean(r)
r_var = np.var(r)

In [390]:
from scipy.sparse import csr_matrix, vstack

In [391]:
D = []
y = []
for m in m_:
    #if m not in ratings:
    #    continue
    d = []
    #d.append(m.num_ratings)
    #d.append((m.mean_rating - r_mean)/r_var)
    #d.append(d[0]*d[1])
    m_g = set()
    if m.genres is not None:
        m_g.update(m.genres)
    m_k = set()
    if m.keywords is not None:
        m_k.update(m.keywords)
    for g in Genres:
        if g in m_g:
            d.append(1)
        else:
            d.append(0)
    for k in Keywords: # VERY INEFFICIENT. FIX!
        if k in m_k:
            d.append(1)
        else:
            d.append(0)
    D.append(csr_matrix(d))
    if m in ratings:
        y.append(1)
    else:
        y.append(0)

In [392]:
len(D)

8000

In [393]:
d = D[0]

In [394]:
for d_ in D[1:]:
    d = vstack([d, d_])

In [395]:
d.shape

(8000, 4233)

In [396]:
len(y)

8000

In [397]:
y = np.array(y)

In [398]:
d[np.where(y==1)]

<51x4233 sparse matrix of type '<class 'numpy.int32'>'
	with 7687 stored elements in Compressed Sparse Row format>

In [399]:
from sklearn import svm

In [400]:
clf = svm.OneClassSVM(kernel='linear')

In [401]:
clf.fit(d[np.where(y==1)])

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='auto',
      kernel='linear', max_iter=-1, nu=0.5, random_state=None,
      shrinking=True, tol=0.001, verbose=False)

In [402]:
clf.coef_

<1x4233 sparse matrix of type '<class 'numpy.float64'>'
	with 3486 stored elements in Compressed Sparse Row format>

In [413]:
indices = np.argsort(np.abs(clf.coef_.toarray()[0]))[::-1]

In [414]:
for i in indices[:100]:
    print(clf.coef_.toarray()[0][i])

25.5
25.5
25.5
21.5347049975
14.9999818054
14.9633048856
13.3581259111
12.0931672238
10.5487109722
10.2340630288
9.87406790135
8.96619374075
8.62582714491
8.57091075671
8.28244541223
8.11864725616
7.99689996747
7.76851197851
7.11821492432
6.78017267695
6.77916597186
6.74940922956
6.74289779646
6.52943648432
6.43279361453
5.93389916553
5.82758266204
5.64209156194
5.58397432056
5.31368606455
4.6833624768
4.64558856232
4.62959055108
4.621592458
4.2799844434
4.27065524056
4.19282433848
4.12133471118
4.08683109051
4.06775774876
3.89032411488
3.87913827393
3.76361889088
3.75770761035
3.75479601338
3.7518883689
3.67316360578
3.62582674696
3.6133109472
3.57930211452
3.48843912118
3.44382507916
3.42765348479
3.4276314545
3.38330256136
3.36528570081
3.36214627458
3.30728170057
3.26575465791
3.25972978887
3.20226537501
3.20097459277
3.19604295099
3.19282433848
3.1917060529
3.15892899497
3.15179335262
3.11909003476
3.11509196745
3.07529076347
3.06842885289
3.06238663635
3.05135566314
3.0510882096


In [405]:
for i in indices[:30]:
    if i < len(Genres):
        print(Genres[i])
    else:
        print(Keywords[i-len(Genres)])

Family
based-on-novel
mother-son-relationship
father-son-relationship
family-relationships
Adventure
Comedy
husband-wife-relationship
character-name-in-title
dog
Fantasy
rescue
Drama
mother-daughter-relationship
Animation
father-daughter-relationship
title-spoken-by-character
surrealism
no-opening-credits
friendship
brother-brother-relationship
brother-sister-relationship
lifting-someone-into-the-air
boy
child's-point-of-view
marriage
blockbuster
school
sequel
second-part


In [406]:
np.where(y==1)

(array([ 965,  975, 1146, 1325, 1432, 1750, 1906, 1931, 1934, 1943, 1944,
        2344, 2345, 2476, 2715, 2731, 3197, 3203, 3310, 3339, 3386, 3606,
        3711, 4327, 4331, 4373, 4378, 4549, 4593, 4697, 4899, 4969, 4984,
        5280, 5539, 5921, 5922, 6000, 6054, 6241, 6449, 6530, 6656, 6675,
        6692, 6907, 7011, 7130, 7294, 7487, 7839], dtype=int64),)

In [407]:
positive = np.where(clf.predict(d) != -1)[0]

In [408]:
len(positive)

69

In [409]:
df = (clf.decision_function(d)).flatten()

In [410]:
indices = np.argsort(df)[::-1]

In [411]:
for i in indices[:10]:
    print(m_[i])
    #print(m_[i].keywords)
    #print()

Charlie and the Chocolate Factory	2005	0000012100	342451	6.7
Nim's Island	2008	0000122000	29128	6.0
King Arthur: Legend of the Sword	2017	0000012211	29921	7.3
Peter Pan	2003	0000012101	56507	6.9
The Dark Knight	2008	0000000124	1792361	9.0
X-Men: Apocalypse	2016	0000012200	275884	7.1
Beauty and the Beast	2017	0000002212	111698	7.6
Hook	1991	0000022100	192345	6.7
Grimsby	2016	0000122100	63445	6.2
Into the Wild	2007	0000001222	459430	8.1


In [378]:
df[indices[:10]]

array([ 367.76355454,  337.05728297,  321.88580193,  317.31529801,
        315.93917039,  297.48398724,  293.41736817,  285.42782701,
        283.76131248,  282.36207652])